<a href="https://colab.research.google.com/github/raphaelcarlosr/linkedin2firebase/blob/main/LinkedIn2Firestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps

1. Load and extract Basic_LinkedInDataExport_xx-xx-xxxx.zip
2. Convert all csv files as json
3. Convert jsons files into json resume scheme
4. Install firebase-admin
5. Update resume into firebase firestore

[Like on github](https://github.com/raphaelcarlosr/linkedin2firebase/blob/main/README.md)



In [50]:
# https://pypi.python.org/pypi/libarchive
# pip install --user firebase-admin
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!pip install --user firebase-admin
!pip install dnspython
!pip install random-object-id
# !pip install --upgrade firebase-admin

import libarchive.public
import pandas
import firebase_admin
import os
import glob
import zipfile
import json
import datetime
from random_object_id import generate
from array import *
# print( pandas.__version__ ) 

clear = lambda: os.system('clear')
clear()

# Set your values
zipped_file = '/content/Complete_LinkedInDataExport_02-19-2021.zip'
credential_file = '/content/drive/MyDrive/Documents/Security/raphaelcarlosr-dev-firebase-adminsdk-bxql2-e5d98b9de5.json' # path to credential json file
output_path = '/content/linkedin/'

# delete all documents in collection
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)


# Extract linkedin data as csv
from zipfile import ZipFile
with ZipFile(zipped_file, 'r') as zipObj:
    zipObj.extractall(output_path)

# Convert all csv as json
csv_files = glob.glob(output_path + "*.csv")
for csv_file in csv_files:
  # print( csv_file )
  name = csv_file.split("/").pop()
  name = name.replace(".csv", "").replace(" ", "-")
  name = name.lower()
  # print( name )
  try:
    df = pandas.read_csv(csv_file, encoding="utf-8")
    # print( df.head(1) ) #View first 10 data rows
    df.to_json( csv_file.replace(".csv", ".json"), orient='records' )
  except FileNotFoundError as e:
    print(e)
  except Exception as e:  
    print("An exception occurred:")
    continue    
     

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_file

clear()

# Set credential
from firebase_admin import credentials
cred = credentials.Certificate(credential_file)

# Import the Firebase service
from firebase_admin import auth

# Initialize the default app
try:
  default_app = firebase_admin.get_app();
except:
  default_app = firebase_admin.initialize_app()

print(default_app.name)    # "[DEFAULT]"

# Setup db
from firebase_admin import firestore
db = firestore.client()
collection = db.collection(u'resume')
invalid_docs = ['resume_rich_media','ad_targeting', "rich_media", "messages", "invitations", "ad_targeting", "ads_clicked", "causes_you_care_about", "comments", "company_follows", "email_addresses", "hashtag_follows", "inferences_about_you", "invitations", "learning", "logins", "meber_follows", "mobile_applications", "phonenumbers", "reactions", "recommendations_given", "registration", "rich_media", "savedjobsalerts", "searchqueries", "shares", "videos", "messages", "savedjobalerts", "contacts", "member_follows"]
id_keys = ["Name", "Degree Name", "Company Name", "First Name", "Title", "First Name|Last Name", "Teammate Name"]
# resume_rich-media
json_files = glob.glob(output_path + "*.json")
for json_file in json_files:
  name = json_file.split("/").pop()
  name = name.replace(".json", "").replace(" ", "-")
  name = name.replace("-", "_")
  name = name.lower()
  try:
    if invalid_docs.index(name) >= 0: #if name == 'ad_targeting':
      continue
  except ValueError:
    pass  

  print(name)
  collection = db.collection(u'resume_' + name)
  delete_collection(collection, 100)
  #docs = collection.list_documents()
  #print(docs)
  
  with open(json_file) as json_content:
      data = json.load(json_content)
      documents = []
      document = {
          # u'_id': '',
          u'dictionary': {},
          u'updatedAt': datetime.datetime.now()
      }
      #doc = {
      #    u'updatedAt': datetime.datetime.now(),
      #    u'values': data
      #}
      for item in data:
        #print(item)
        keys = item.keys()
        id = generate()
        #print(keys)
        for key in keys:
          #print(key)
          document[key] = item[key]
          try:
            index = id_keys.index(key)
            if index >= 0:
              id = (document[id_keys[index]]).replace('/', '-')
              #print(id)                        
          except ValueError:
            pass
        
        print("ID = " + id)    
        document['_id'] = id #generate() #ObjectId(generate())
        document['dictionary'] = { 
            u'pt': {}
        }
        for key in keys:
          document['dictionary']['pt'][key] = item[key]
        
        print(document)
        collection.document(id).set(document, merge=False)
        documents.append(documents)

      #collection.document(name).set(document, merge=True)
  #break



# Remove files
files = glob.glob(output_path + "*.csv") +  glob.glob(output_path + "*.json") +  glob.glob(output_path + 'Jobs/**/*.csv', recursive=True) +  glob.glob(output_path + 'Jobs/**/*.json', recursive=True)
# print (files)
for f in files:
    try:
        # f.unlink()
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))

os.rmdir(output_path + 'Jobs/Job Application Answers')
os.rmdir(output_path + 'Jobs')
os.rmdir(output_path)





Requirement already up-to-date: libarchive in /usr/local/lib/python3.7/dist-packages (0.4.7)
An exception occurred:
[DEFAULT]
courses
Deleting doc JavaScript => {'dictionary': {'pt': {'Name': 'JavaScript', 'Number': '2009 - Impacta'}}, '_id': 'JavaScript', 'updatedAt': DatetimeWithNanoseconds(2021, 6, 7, 20, 36, 9, 276934, tzinfo=<UTC>), 'Number': '2009 - Impacta', 'Name': 'JavaScript'}
Deleting doc MS Project 2016 VS PMBOOK - Ciclo de vida => {'Name': 'MS Project 2016 VS PMBOOK - Ciclo de vida', 'dictionary': {'pt': {'Name': 'MS Project 2016 VS PMBOOK - Ciclo de vida', 'Number': '2016'}}, '_id': 'MS Project 2016 VS PMBOOK - Ciclo de vida', 'updatedAt': DatetimeWithNanoseconds(2021, 6, 7, 20, 36, 9, 276934, tzinfo=<UTC>), 'Number': '2016'}
Deleting doc PHP & MySQL Web Developer => {'updatedAt': DatetimeWithNanoseconds(2021, 6, 7, 20, 36, 9, 276934, tzinfo=<UTC>), '_id': 'PHP & MySQL Web Developer', 'Number': '2007 - 4Linux', 'Name': 'PHP & MySQL Web Developer', 'dictionary': {'pt': {'N